##LIbrerias

In [3]:
!pip install python-doctr

In [4]:
!pip install tf2onnx PyMuPDF pandas opencv-python-headless pdf2image --upgrade Pillow

##Code

In [21]:
#import shutil
#shutil.rmtree('/content/output')
#shutil.rmtree('/content/tiles')

In [ ]:
#!python cutpdf_subprograma.py
#!python passtheocr.py

In [5]:
import fitz  # PyMuPDF
import pandas as pd
import csv
import subprocess
import io
from google.colab.patches import cv2_imshow
import cv2
import numpy as np
import math
import os
from doctr.io import DocumentFile
from doctr.models import ocr_predictor
from PIL import Image, ImageDraw
from typing import Any, List, Tuple

In [24]:
def get_pdf_size(path, tolerance=0.10):
    # Open the PDF file
    pdf = fitz.open(path)

    # Create a dictionary to store the count of each page size
    page_size_counts = {}

    # Iterate through each page in the PDF
    for page_number in range(pdf.page_count):
        page = pdf[page_number]
        width, height = page.rect.width, page.rect.height

        # Define the sizes in points with a tolerance
        sizes = {
            'A0': (2384, 3370),
            'A1': (1684, 2384),
            'A2': (1191, 1684),
            'A3': (842, 1191),
            'A4': (595, 842),
        }

        # Compare the dimensions to determine the size within the tolerance
        for size, dims in sizes.items():
            if (
                (abs(width - dims[0]) <= dims[0] * tolerance and abs(height - dims[1]) <= dims[1] * tolerance)
                or (abs(width - dims[1]) <= dims[1] * tolerance and abs(height - dims[0]) <= dims[0] * tolerance)
            ):
                # Increment the count for the current page size
                page_size_counts[size] = page_size_counts.get(size, 0) + 1

    # Find the most repeated page size
    most_repeated_size = max(page_size_counts, key=page_size_counts.get)

    return most_repeated_size, page_size_counts, pdf.page_count, pdf  # Include the PDF object in the return

def classify_document(pdf_path, csv_path):
    # Check if it's a drawing
    most_repeated_size, page_size_counts, num_pages, pdf = get_pdf_size(pdf_path)

    if most_repeated_size == 'A4':
        return f"This is a PDF-Document, Most Repeated Size: {most_repeated_size}, Page Size Counts: {page_size_counts}, Number of Pages: {num_pages}."
    else:
        # Run the subprogram if it's a drawing
        subprocess.run(['python', 'cutpdf_subprograma.py'])

        subprocess.run(['python', 'passtheocr.py'])
        result = subprocess.run(['python', 'passtheocr.py'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

        # Read the original CSV file
        original_df = pd.read_csv(csv_path)

        # Create a new DataFrame with 'tile_num' and 'text' columns
        new_df = pd.DataFrame(columns=['tile_num', 'text'])

        # Iterate over unique tile numbers and concatenate all texts consecutively
        for tile_num in original_df['tile_num'].unique():
            # Filter rows for the current tile number
            rows_for_tile = original_df[original_df['tile_num'] == tile_num]

            # Concatenate all texts consecutively
            concatenated_text = ' '.join(rows_for_tile['text'].astype(str))

            # Append a new row to the new DataFrame
            new_df = pd.concat([new_df, pd.DataFrame({'tile_num': [tile_num], 'text': [concatenated_text]})], ignore_index=True)

        # Save the new DataFrame to a new CSV file
        new_df.to_csv('modified_csv_path.csv', index=False)

        df = pd.read_csv(csv_path_concatenated)

        keywords = [
            'electrical', 'electric', 'wiring', 'wiring diagram',
            'eléctrico', 'eléctrica', 'cableado',
            'circuito', 'circuit'
        ]
        is_electrical_diagram = any(df['text'].str.contains('|'.join(keywords), case=False, na=False))

        # Additional classification for PDF-P&ID
        pid_keywords = ['P&ID', 'PID']  # Add more P&ID-related keywords here
        is_pid_document = any(df['text'].str.contains('|'.join(pid_keywords), case=False, na=False))

        # Additional classification for PDF-Drawing Isometric
        drawing_keywords = ['VISTA ISOMÉTRICA', 'VISTA ISOMÉTRICO', 'ISOMÉTRICO', 'ISOMÉTRICA']  # Add more drawing-related keywords here
        is_drawing_Isometric = any(df['text'].str.contains('|'.join(drawing_keywords), case=False, na=False))

        # Additional classification for PDF-Drawing General Arrangement
        arragment_keywords = ['VISTA FRONTAL', 'VISTA PLANTA']  # Add more drawing-related keywords here
        is_drawing_General = any(df['text'].str.contains('|'.join(arragment_keywords), case=False, na=False))

        if is_pid_document:
            document_type = 'PDF-P&ID'
        elif is_electrical_diagram:
            document_type = 'PDF-Electrical Diagram'
        elif is_drawing_Isometric:
            document_type = 'PDF-Drawing Isometric'
        elif is_drawing_General:
            document_type = 'PDF-Drawing General Arragment'
        else:
            document_type = 'PDF-Drawing Others'

        # Extract the PDF name from the pdf_path
        pdf_name = os.path.basename(pdf_path)

        # Close the PDF file after extracting information
        pdf.close()

        return f"This is a {document_type}-{pdf_name}, Most Repeated Size: {most_repeated_size}, Page Size Counts: {page_size_counts}, Number of Pages: {num_pages}."

# Example usage:
pdf_path = '2.pdf'
csv_path = '/content/final_output.csv'
csv_path_concatenated = '/content/modified_csv_path.csv'

result_message = classify_document(pdf_path, csv_path)
print(result_message)

This is a PDF-Document, Most Repeated Size: A4, Page Size Counts: {'A4': 20}, Number of Pages: 20.
